In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import keplergl
#from keplergl import KeplerGL
import matplotlib as plt
import plotly.express as px
import h3
import shapely
from geopandas import GeoDataFrame,GeoSeries
import folium
import matplotlib
%matplotlib inline

/Users/rakeenahmed/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [152]:
bikelanes = gpd.read_file('cycling-network.geojson')

In [153]:
type(bikelanes)

geopandas.geodataframe.GeoDataFrame

In [154]:
bikelanes.head()

,_id,OBJECTID,SEGMENT_ID,INSTALLED,UPGRADED,PRE_AMALGAMATION,STREET_NAME,FROM_STREET,TO_STREET,ROADCLASS,...,INFRA_HIGHORDER,SEPA_HIGHORDER,SEPB_HIGHORDER,ORIG_HIGHORDER,BYLAWED,EDITOR,LAST_EDIT_DATE,UPGRADE_DESCRIPTION,CONVERTED,geometry
0,1,1,1,2001,2021,,Kilbarry Rd,Highbourne Rd,Oriole Pkwy,,...,Sharrows - Wayfinding,,,,,,2023-01-16 19:53:01,,2007,"MULTILINESTRING ((-79.40351 43.69526, -79.4030..."
1,2,2,2,2001,2009,,Martin Goodman / Waterfront Trl,Lake Shore Blvd W,Martin Goodman Trl Branch,,...,Multi-Use Trail,,,,,,2023-01-16 19:53:01,,,"MULTILINESTRING ((-79.40364 43.63499, -79.4035..."
2,3,3,3,2001,2011,,Gatineau Hydro Corridor Trl,Givendale Rd,Kennedy Rd,,...,Multi-Use Trail,,,,,,2023-01-16 19:53:01,,,"MULTILINESTRING ((-79.27522 43.74158, -79.2751..."
3,4,4,4,2001,2011,,G Ross Lord Park Trl,Dufferin St,G Ross Lord Park Trl Branch,,...,Multi-Use Trail,,,,,,2023-01-16 19:53:01,,,"MULTILINESTRING ((-79.46772 43.77208, -79.4675..."
4,5,5,5,2001,2012,,Etobicoke Creek Trl,Etobicoke Creek Trl Branch,Forty Third St,,...,Multi-Use Trail - Entrance,,,,,,2023-01-16 19:53:01,,,"MULTILINESTRING ((-79.54812 43.59027, -79.5480..."


In [155]:
bikelanes.columns

Index(['_id', 'OBJECTID', 'SEGMENT_ID', 'INSTALLED', 'UPGRADED',
       'PRE_AMALGAMATION', 'STREET_NAME', 'FROM_STREET', 'TO_STREET',
       'ROADCLASS', 'CNPCLASS', 'SURFACE', 'OWNER', 'DIR_LOWORDER',
       'INFRA_LOWORDER', 'SEPA_LOWORDER', 'SEPB_LOWORDER',
       'ORIG_LOWORDER_INFRA', 'DIR_HIGHORDER', 'INFRA_HIGHORDER',
       'SEPA_HIGHORDER', 'SEPB_HIGHORDER', 'ORIG_HIGHORDER', 'BYLAWED',
       'EDITOR', 'LAST_EDIT_DATE', 'UPGRADE_DESCRIPTION', 'CONVERTED',
       'geometry'],
      dtype='object')

In [156]:
#shape of dataframe
bikelanes.shape

(1323, 29)

In [157]:
#converting dataframe headers to lowercase
bikelanes.columns = [s.strip().lower() for s in bikelanes.columns]

In [158]:
#drop unneeded columns
bikelanes = bikelanes.drop(columns=['objectid','segment_id','upgraded','pre_amalgamation','roadclass','cnpclass','surface','owner','dir_loworder','sepa_loworder','sepb_loworder','orig_loworder_infra','dir_highorder','sepa_highorder','sepb_highorder','orig_highorder','bylawed','editor','last_edit_date','upgrade_description','converted'])

In [159]:
#rename id column
bikelanes = bikelanes.rename(columns={'_id':'id'})

In [160]:
bikelanes.head()

,id,installed,street_name,from_street,to_street,infra_loworder,infra_highorder,geometry
0,1,2001,Kilbarry Rd,Highbourne Rd,Oriole Pkwy,Sharrows - Wayfinding,Sharrows - Wayfinding,"MULTILINESTRING ((-79.40351 43.69526, -79.4030..."
1,2,2001,Martin Goodman / Waterfront Trl,Lake Shore Blvd W,Martin Goodman Trl Branch,Multi-Use Trail,Multi-Use Trail,"MULTILINESTRING ((-79.40364 43.63499, -79.4035..."
2,3,2001,Gatineau Hydro Corridor Trl,Givendale Rd,Kennedy Rd,Multi-Use Trail,Multi-Use Trail,"MULTILINESTRING ((-79.27522 43.74158, -79.2751..."
3,4,2001,G Ross Lord Park Trl,Dufferin St,G Ross Lord Park Trl Branch,Multi-Use Trail,Multi-Use Trail,"MULTILINESTRING ((-79.46772 43.77208, -79.4675..."
4,5,2001,Etobicoke Creek Trl,Etobicoke Creek Trl Branch,Forty Third St,Multi-Use Trail - Entrance,Multi-Use Trail - Entrance,"MULTILINESTRING ((-79.54812 43.59027, -79.5480..."


In [161]:
bikelanes.dtypes

id                    int64
installed             int64
street_name          object
from_street          object
to_street            object
infra_loworder       object
infra_highorder      object
geometry           geometry
dtype: object

In [162]:
from keplergl import KeplerGl

KeplerGl(data={'data_1':bikelanes})

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'data_1':         id  installed                      street_name  \
0        1       2001      …

In [163]:
#using the geopandas explode() function to convert the multiline geometry to single line point geometry
#bikelanes_exploded = bikelanes.explode()

In [164]:
#extract latitude and longitudes from single line geometry
#bikelanes_exploded['latitude'] = bikelanes_exploded.apply(lambda x: x.geometry.coords[0][1], axis=1)
#bikelanes_exploded['longitude'] = bikelanes_exploded.apply(lambda x: x.geometry.coords[0][0], axis=1)

In [165]:
#alternate method: extract latitude and longitude from centre of linestring geometry
#bikelanes_exploded['latitude'] = bikelanes_exploded.apply(lambda x: x.geometry.centroid.coords[0][1], axis=1)
#bikelanes_exploded['longitude'] = bikelanes_exploded.apply(lambda x: x.geometry.centroid.coords[0][0], axis=1)

In [166]:
#combining latitude and longitudes to form coordinates column
#bikelanes_exploded['coordinates'] = bikelanes_exploded.apply(lambda x: (x.latitude, x.longitude), axis=1)

In [167]:
#create hex cells for bike lanes

#create coordinate column
#pedestrian_crossover_exploded['coordinates'] = pedestrian_crossover_exploded['latitude'].astype('str') + ',' + pedestrian_crossover_exploded['longitude'].astype('str')

#then we define the function to get h3 hex cells from coordinates data
#def get_hex_cell(coordinates):
    #a = float(coordinates.split(',')[0])
    #b = float(coordinates.split(',')[1])
    a = coordinates[0]
    b = coordinates[1]
    return h3.latlng_to_cell(a,b,res=10)

#finally, create a new column for hex cells by applying the function on coordinates column
#bikelanes_exploded['hex_cell'] = bikelanes_exploded['coordinates'].apply(get_hex_cell)

IndentationError: unexpected indent (195119267.py, line 10)

In [168]:
#check final dataframe
#bikelanes_exploded.head()

In [169]:
bikelanes.columns

Index(['id', 'installed', 'street_name', 'from_street', 'to_street',
       'infra_loworder', 'infra_highorder', 'geometry'],
      dtype='object')

In [170]:
type(bikelanes)

geopandas.geodataframe.GeoDataFrame

In [171]:
#final processing

#drop unneeded columns
bikelanes = bikelanes.drop(columns=['installed','from_street','to_street','infra_loworder','infra_highorder'])

#set id as index
bikelanes.set_index('id', inplace=True)

#adding a new column as a flag for bike lane = "Yes"
bikelanes['bikelane'] = 'Yes'

In [172]:
#adding column prefix to all columns
bikelanes.columns = ['bikelanes_' + s if s != 'geometry' else s for s in bikelanes.columns]

In [173]:
type(bikelanes)

pandas.core.frame.DataFrame

In [174]:
bikelanes.to_csv('bikelanes_processed.csv')

### Creating another dataframe to extract hex cells from bikelanes centroid geometry

In [48]:
bikelanes_exploded_2 = bikelanes.explode()

#alternate method: extract latitude and longitude from centre of linestring geometry
bikelanes_exploded_2['latitude'] = bikelanes_exploded_2.apply(lambda x: x.geometry.centroid.coords[0][1], axis=1)
bikelanes_exploded_2['longitude'] = bikelanes_exploded_2.apply(lambda x: x.geometry.centroid.coords[0][0], axis=1)

#combining latitude and longitudes to form coordinates column
bikelanes_exploded_2['coordinates'] = bikelanes_exploded_2.apply(lambda x: (x.latitude, x.longitude), axis=1)

#finally, create a new column for hex cells by applying the function on coordinates column
bikelanes_exploded_2['hex_cell'] = bikelanes_exploded_2['coordinates'].apply(get_hex_cell)

#final processing

#set id as index
bikelanes_exploded_2.set_index('id', inplace=True)

#drop geometry, latitude, and longitudes columns
bikelanes_exploded_2.drop(columns=['geometry','latitude','longitude'], inplace=True)

C:\Users\arara\AppData\Local\Temp\ipykernel_36368\3145448852.py:1: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  bikelanes_exploded_2 = bikelanes.explode()


In [49]:
bikelanes_exploded_2.head()

,installed,street_name,from_street,to_street,infra_loworder,infra_highorder,coordinates,hex_cell
id,,,,,,,,
1,2001,Kilbarry Rd,Highbourne Rd,Oriole Pkwy,Sharrows - Wayfinding,Sharrows - Wayfinding,"(43.69537795963702, -79.40295543460897)",8a2b9bc76357fff
2,2001,Martin Goodman / Waterfront Trl,Lake Shore Blvd W,Martin Goodman Trl Branch,Multi-Use Trail,Multi-Use Trail,"(43.635383460222734, -79.40341281112208)",8a2b9bc42d8ffff
3,2001,Gatineau Hydro Corridor Trl,Givendale Rd,Kennedy Rd,Multi-Use Trail,Multi-Use Trail,"(43.74214616290669, -79.2737540413134)",8a2b9bd4edaffff
4,2001,G Ross Lord Park Trl,Dufferin St,G Ross Lord Park Trl Branch,Multi-Use Trail,Multi-Use Trail,"(43.77358708102622, -79.46748984968795)",8a2b9bc04b67fff
5,2001,Etobicoke Creek Trl,Etobicoke Creek Trl Branch,Forty Third St,Multi-Use Trail - Entrance,Multi-Use Trail - Entrance,"(43.59016644008738, -79.5474509580628)",8a2b9bcea6affff


Concatenating the two dataframes

In [50]:
bikelanes_processed = pd.concat([bikelanes_exploded,bikelanes_exploded_2], ignore_index=True)

In [51]:
bikelanes_processed.head()

,installed,street_name,from_street,to_street,infra_loworder,infra_highorder,coordinates,hex_cell
0,2001,Kilbarry Rd,Highbourne Rd,Oriole Pkwy,Sharrows - Wayfinding,Sharrows - Wayfinding,"(43.6952595244941, -79.4035069136297)",8a2b9bc76357fff
1,2001,Martin Goodman / Waterfront Trl,Lake Shore Blvd W,Martin Goodman Trl Branch,Multi-Use Trail,Multi-Use Trail,"(43.6349938275931, -79.4036386937994)",8a2b9bc42ca7fff
2,2001,Gatineau Hydro Corridor Trl,Givendale Rd,Kennedy Rd,Multi-Use Trail,Multi-Use Trail,"(43.7415816548541, -79.2752236011477)",8a2b9bd4ed8ffff
3,2001,G Ross Lord Park Trl,Dufferin St,G Ross Lord Park Trl Branch,Multi-Use Trail,Multi-Use Trail,"(43.7720763078272, -79.4677191149987)",8a2b9bc04b6ffff
4,2001,Etobicoke Creek Trl,Etobicoke Creek Trl Branch,Forty Third St,Multi-Use Trail - Entrance,Multi-Use Trail - Entrance,"(43.5902679719901, -79.5481155366534)",8a2b9bcea6affff


In [53]:
bikelanes_processed.shape

(3112, 8)

In [55]:
bikelanes_exploded.hex_cell.nunique()

1132

In [56]:
bikelanes_exploded_2.hex_cell.nunique()

1368

In [57]:
bikelanes_processed.hex_cell.nunique()

2045

In [58]:
#save processed dataframe as csv
bikelanes_processed.to_csv('bikelanes_processed.csv')